# DeepFM

Eileen Zhang 2020/8/20

![deepfm](../data/deepfm.png)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
from tensorflow.keras.layers import Layer,Dense,Embedding,Dropout,BatchNormalization

In [3]:
from tensorflow.keras.models import Model

In [4]:
class MultiLayerPerceptron(Layer):
    def __init__(self, units, dropout = 0.8, output_layer=True):
        super().__init__()
        self.output_layer = output_layer
        self.dropout = Dropout(dropout)
        self.bns = [BatchNormalization() for i in range(len(units))]
        self.denses = [Dense(u,activation = 'relu') for u in units]
        if output_layer:
            self.denses.append(Dense(1))
     
    def build(self, input_shape):    
        super().build(input_shape)
    
    def call(self, inputs, training=False):
        for bn, dense in zip(self.bns,self.denses) :
            inputs = bn(inputs,training)
            inputs = dense(inputs)
            if training:
                inputs = self.dropout_layer(inputs, training=training)
        if self.output_layer:
            inputs = self.denses[-1](inputs)
        return inputs

In [5]:
class FieldAwareFactorizationMachine(Layer):
    import tensorflow as tf

    def __init__(self, field_dim_groups, embed_dim):
        """
        :param x: field_dim_groups features group(the max_len for features)eg: [[10,20,40],[20,30,10],[5,6]]]``
                  num_field_groups : [0,3,6]``
        
        """        
        super().__init__()
        self.field_dim_groups = field_dim_groups
        self.features_len = np.sum([len(x) for x in field_dim_groups])
        self.num_field_groups = np.cumsum([0] + [len(x) for x in field_dim_groups])[:-1]
        #for irregular list [[10,20,40],[20,30,10],[5,6]]]
        self.maxlen = np.sum(np.sum(field_dim_groups))
        self.embeddings = Embedding(self.maxlen, embed_dim)
        self.offsets = tf.constant(np.expand_dims(np.array((0, *np.cumsum([x for y in field_dim_groups for x in y])[:-1]), dtype=np.float32),0))

    def build(self, input_shape):    
        super().build(input_shape)
        
    def call(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + self.offsets
        x = self.embeddings(x)
        ix = []
        for i in range(len(self.field_dim_groups) - 1):           
            for j in range(self.num_field_groups[i], self.num_field_groups[i + 1]):
                ix += [x[:, j, : ] * x[:, k, :] for k in range(self.num_field_groups[i + 1], self.features_len)]
        ix = tf.stack(ix)
        ix = tf.transpose(ix, perm=[1, 0, 2])
        return tf.concat([x, ix],1), x

In [6]:
class DeepFactorizationMachineModel(Model):
    """
    A keras implementation of DeepFM.

    Reference:
        H Guo, et al. DeepFM: A Factorization-Machine based Neural Network for CTR Prediction, 2017.
    """

    def __init__(self, field_dim_groups, embed_dim, units = [100,50]):
        super().__init__()
        units = units + [embed_dim]
        self.embed_dim = embed_dim
        self.ffm = FieldAwareFactorizationMachine(field_dim_groups, embed_dim)
        self.mlp = MultiLayerPerceptron(units,output_layer = False)
        self.dense = Dense(1)
        self.fc = Dense(1, activation='sigmoid')
        
    def call(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x_ffm, x = self.ffm(x)
        x_mlp = self.mlp(x)     
        out = tf.concat([x_ffm,x_mlp],1)
        out = tf.squeeze(self.dense(out),-1)
        out = self.fc(out)
        return out

In [7]:
model = DeepFactorizationMachineModel([[10,20,40],[50,51]],3)

In [8]:
test = tf.constant([[0.,1.,2.,3.,4.],[10.,20.,30.,40.,50.]])
test

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[ 0.,  1.,  2.,  3.,  4.],
       [10., 20., 30., 40., 50.]], dtype=float32)>

In [9]:
model(test)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.504937  ],
       [0.49564382]], dtype=float32)>

In [10]:
model.summary()

Model: "deep_factorization_machine_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
field_aware_factorization_ma multiple                  513       
_________________________________________________________________
multi_layer_perceptron (Mult multiple                  6215      
_________________________________________________________________
dense_3 (Dense)              multiple                  4         
_________________________________________________________________
dense_4 (Dense)              multiple                  17        
Total params: 6,749
Trainable params: 6,443
Non-trainable params: 306
_________________________________________________________________
